# Рынок заведений общественного питания Москвы

## Описание проекта

Вы решили открыть небольшое кафе в Москве. Оно оригинальное — гостей должны обслуживать роботы. Проект многообещающий, но дорогой. Вместе с партнёрами вы решились обратиться к инвесторам. Их интересует текущее положение дел на рынке — сможете ли вы снискать популярность на долгое время, когда все зеваки насмотрятся на роботов-официантов? Партнёры просят вас подготовить исследование рынка. У вас есть открытые данные о заведениях общественного питания в Москве.

**Сделайте общий вывод и дайте рекомендации о виде заведения, количестве посадочных мест, а также районе расположения. Прокомментируйте возможность развития сети. Выводы подготовьте в формате PDF-презентации.**

## Структура проекта

1. [Знакомство с данными](#1)
2. [Предобработка](#2)
3. [Вычленение улицы из адреса. RegEx](#3)
4. [Получение геоданных по адресу. Работа с API](#4)
5. [Обогащение данных информацией о населении районов](#5)
6. [EDA](#6)
7. [Результаты исследования](#7)

## Прекод

In [1]:
!pip install plotly -U
!pip install pymorphy2 -U
!pip install geocoder -U

     |████████████████████████████████| 55 kB 3.5 MB/s 
     |████████████████████████████████| 8.2 MB 14.7 MB/s 
     |████████████████████████████████| 98 kB 9.5 MB/s 


In [2]:
# игнорируем предупреждения
import warnings
warnings.filterwarnings('ignore')

# работа с таблицамими, вычисления
import numpy as np
import pandas as pd
from scipy import stats as st

# регулярки
import re 

# для импорта файла из Google Sheets
import requests
from io import BytesIO

# для приведения названия районов к нормальной форме для мерджа таблиц
import pymorphy2

# Google Geocoding API, парсинг геоданных
import geocoder

# для приватного хранения токена
import os


# визуализация
import matplotlib.pyplot as plt
import seaborn as sns
import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# параметры визуализации
sns.set()
sns.set_context(
    "notebook", 
    font_scale=1.3,       
    rc={ 
        "figure.figsize": (12, 10), 
        "axes.titlesize": 18 
    }
)
from matplotlib import rcParams
rcParams['figure.figsize'] = 12, 10

# параметры отображения таблиц
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

## 1. Знакомство с данными <a name="1"></a>

In [3]:
LINK = ''
PRAKTIKUM_DIR = '/datasets/'

REST_DATA = 'rest_data.csv'

In [4]:
try:
    raw_df = pd.read_csv(PRAKTIKUM_DIR+REST_DATA)
    print('Файлы прочитаны. Работаем в среде Я.Практикума')
except:
    raw_df = pd.read_csv(LINK+REST_DATA)
    print('Данные прочитаны. Работаем вне среды Я.Практикума')

Данные прочитаны. Работаем вне среды Я.Практикума


In [5]:
print('Размерность raw_df:', raw_df.shape)

Размерность raw_df: (15366, 6)


In [6]:
raw_df.sample(10, random_state=42)

,id,object_name,chain,object_type,address,number
12062,182973,БУРГЕР КИНГ,да,кафе,"город Москва, Ярославское шоссе, дом 69",60
8819,156321,Кафе «Тетя Мотя»,нет,кафе,"город Москва, проезд Сокольнического Круга, дом 7",100
14197,206042,Чебуречная,нет,магазин (отдел кулинарии),"город Москва, Открытое шоссе, дом 9, строение 9",0
6081,156891,Чудо Пекарня Шаргунь,нет,закусочная,"город Москва, Севастопольский проспект, дом 15, корпус 3",0
2478,27730,Столовая «ГАЛА ПИЦЦА»,нет,столовая,"город Москва, Варшавское шоссе, дом 116",46
1370,154280,Пиво-воды,нет,бар,"город Москва, Щёлковское шоссе, дом 85, корпус 1",22
12664,201989,Пекарня,нет,закусочная,"город Москва, Липецкая улица, дом 50, строение 2",0
14930,223189,Porto-Pomodoro,нет,ресторан,"город Москва, Ходынский бульвар, дом 4",0
3889,25948,Форбес,нет,ресторан,"город Москва, Подколокольный переулок, дом 13, строение 1",88
6338,152835,Диктатура Эстетика,нет,кафе,"город Москва, Берсеневская набережная, дом 6, строение 2",38


**Описание столбцов**

* id — идентификатор объекта;
* object_name — название объекта общественного питания;
* chain — сетевой ресторан;
* object_type — тип объекта общественного питания;
* address — адрес;
* number — количество посадочных мест.

Посмотрим на распределение уникальных значений в столбцах

In [7]:
for col in raw_df.columns:
    display(raw_df[col].value_counts())

163840    1
207007    1
58806     1
198074    1
150035    1
         ..
29419     1
69506     1
23278     1
185071    1
167934    1
Name: id, Length: 15366, dtype: int64

Столовая           267
Кафе               236
Шаурма             234
KFC                155
Шоколадница        142
                  ... 
Мама, я в лапшу      1
Кафе «Лайк»          1
АРИАДНА              1
Апрель               1
МУГАН                1
Name: object_name, Length: 10393, dtype: int64

нет    12398
да      2968
Name: chain, dtype: int64

кафе                                 6099
столовая                             2587
ресторан                             2285
предприятие быстрого обслуживания    1923
бар                                   856
буфет                                 585
кафетерий                             398
закусочная                            360
магазин (отдел кулинарии)             273
Name: object_type, dtype: int64

город Москва, Ходынский бульвар, дом 4                           95
город Москва, Пресненская набережная, дом 2                      63
город Москва, проспект Мира, дом 211, корпус 2                   60
город Москва, Кировоградская улица, дом 13А                      53
город Москва, площадь Киевского Вокзала, дом 2                   48
                                                                 ..
город Москва, проспект 60-летия Октября, дом 23                   1
город Москва, Сколковское шоссе, дом 32, корпус 1                 1
город Москва, Варшавское шоссе, дом 144, корпус 2, строение 2     1
город Москва, улица Образцова, дом 9, строение 4                  1
город Москва, Осенний бульвар, дом 9                              1
Name: address, Length: 9108, dtype: int64

0      1621
40      835
20      727
30      685
10      644
       ... 
172       1
520       1
680       1
760       1
495       1
Name: number, Length: 315, dtype: int64

In [8]:
raw_df['number'].describe()

count    15366.000000
mean        59.547182
std         74.736833
min          0.000000
25%         12.000000
50%         40.000000
75%         80.000000
max       1700.000000
Name: number, dtype: float64

In [9]:
raw_df.query('number == 0').shape[0] / raw_df.shape[0]

0.10549264610178316

### Вывод

- имеем разные регистры в значениях столбцов `object_name` и `adress`. Нужно привести к единому регистру для корректного учета дубликатов и уникальных значений.
- столбец `address` потребует выделения названия улицы из полного адреса для анализа популярных улиц согласно заданию.
- типы данных корректны, пропусков нет.
- дубликаты проверим на этапе предобработки.
- около 10.5% записей-ресторанов имеют 0 посадных мест. Нули могут быть характерны для заведений с 'едой на вынос'. Не будем считать это ошибкой в данных.

## 2. Предобработка <a name="2"></a>

Делаем копию изначальных данных. Приводим значения столбцов `object_name` и `adress` к единому нижнему регистру.

In [10]:
df = raw_df.copy()

In [11]:
df[['object_name', 'address']] = raw_df[['object_name', 'address']].applymap(lambda x: x.lower())

Сравним результат `.value_counts()` для этих столбцов: их изначальный вид VS приведенный к нижнему регистру.

In [12]:
display(raw_df['object_name'].value_counts().head(10))
display(df['object_name'].value_counts().head(10))

Столовая           267
Кафе               236
Шаурма             234
KFC                155
Шоколадница        142
Макдоналдс         122
Бургер Кинг        122
Домино'с Пицца      86
Теремок             84
Крошка Картошка     82
Name: object_name, dtype: int64

столовая           321
кафе               278
шаурма             250
шоколадница        158
kfc                155
макдоналдс         151
бургер кинг        137
теремок             94
крошка картошка     90
домино'с пицца      90
Name: object_name, dtype: int64

In [13]:
display(raw_df['address'].value_counts().head(10))
display(df['address'].value_counts().head(10))

город Москва, Ходынский бульвар, дом 4            95
город Москва, Пресненская набережная, дом 2       63
город Москва, проспект Мира, дом 211, корпус 2    60
город Москва, Кировоградская улица, дом 13А       53
город Москва, площадь Киевского Вокзала, дом 2    48
город Москва, улица Земляной Вал, дом 33          46
город Москва, Мытная улица, дом 74                46
город Москва, улица Новый Арбат, дом 21           42
город Москва, улица Ленинская Слобода, дом 26     41
город Москва, Ярцевская улица, дом 19             40
Name: address, dtype: int64

город москва, ходынский бульвар, дом 4            95
город москва, пресненская набережная, дом 2       63
город москва, проспект мира, дом 211, корпус 2    60
город москва, кировоградская улица, дом 13а       53
город москва, площадь киевского вокзала, дом 2    48
город москва, улица земляной вал, дом 33          46
город москва, мытная улица, дом 74                46
город москва, улица новый арбат, дом 21           42
город москва, улица ленинская слобода, дом 26     41
город москва, кутузовский проспект, дом 57        40
Name: address, dtype: int64

- приведение названий объектов питания `object_name` к нижнему регистру действительно помогло убрать дубликаты.
- однако в случае со столбцом `address` ничего не изменилось. Вернем этому столбцу изначальные адреса с наличием заглавных букв для лучшей читабельности.

In [14]:
df['address'] = raw_df['address']

In [15]:
df.head()

,id,object_name,chain,object_type,address,number
0,151635,сметана,нет,кафе,"город Москва, улица Егора Абакумова, дом 9",48
1,77874,родник,нет,кафе,"город Москва, улица Талалихина, дом 2/1, корпус 1",35
2,24309,кафе «академия»,нет,кафе,"город Москва, Абельмановская улица, дом 6",95
3,21894,пиццетория,да,кафе,"город Москва, Абрамцевская улица, дом 1",40
4,119365,кафе «вишневая метель»,нет,кафе,"город Москва, Абрамцевская улица, дом 9, корпус 1",50


Дубликаты будем искать по строке за исключением столбца `id`, т.к. все айдишники уникальны, значит поиск дубликатов с этим столбцом ничего не даст. Также проверим дубликаты без учета кол-ва посадочных мест `number`.

In [16]:
print('Количество полных дубликатов:', df.duplicated().sum())
print('Количество дубликатов без столбца id:', df.iloc[:, 1:].duplicated().sum())
print('Количество дубликатов без столбцов id и number:', df.iloc[:, 1:-1].duplicated().sum())


Количество полных дубликатов: 0
Количество дубликатов без столбца id: 85
Количество дубликатов без столбцов id и number: 183


Второй вариант показал больше дубликатов, значит у ряда заведений полностью совпадает название, категория, адрес, но различается количество посадочных мест. Возможно ли такое в реальности? Вряд ли. Посмотрим на эти строки.

In [17]:
df[df.iloc[:, 1:-1].duplicated()].sample(10, random_state=42)

,id,object_name,chain,object_type,address,number
4462,26394,центр павла слободкина,нет,буфет,"город Москва, улица Арбат, дом 48, строение 1",55
5393,71373,закусочная kfc,да,предприятие быстрого обслуживания,"город Москва, проспект Мира, дом 211, корпус 2",86
14204,207883,пунк питания,нет,кафе,"город Москва, Большая Черкизовская улица, дом 125",0
12193,192574,каменев василий владимирович,нет,кафе,"город Москва, улица Сущёвский Вал, дом 5, строение 12",2
13216,194256,кафе «white cafe»,нет,кафе,"город Москва, Краснопресненская набережная, дом 12",8
3481,20914,буфет мади,нет,буфет,"город Москва, Ленинградский проспект, дом 64",40
5060,20015,lift,нет,кафе,"город Москва, Багратионовский проезд, дом 7, корпус 1",40
6537,23787,столовая при мгу,нет,столовая,"город Москва, территория Ленинские Горы, дом 1",40
12734,192923,lift,нет,кафе,"город Москва, Багратионовский проезд, дом 7, корпус 3",3
11372,190471,буфет,нет,буфет,"город Москва, Пятницкое шоссе, дом 18",0


Удаляем эти 183 строки. Для повторающихся строк-дублей оставляем только одну уникальную запись.

In [18]:
df.columns[1:-1].to_list()

['object_name', 'chain', 'object_type', 'address']

In [19]:
df = df.drop_duplicates(subset=df.columns[1:-1].to_list())

In [20]:
print(raw_df.shape[0])
print(df.shape[0])

15366
15183


## 3. Вычленение улицы из адреса. RegEx <a name="3"></a>

Теперь выделим названия улиц из адресов. Посмотрим на выборку адресов.

In [21]:
display(df['address'].sample(30, random_state=42))

12535                           Вознесенский переулок, дом 7, строение 1
11343                           город Москва, Бартеневская улица, дом 59
6237                                город Москва, Венёвская улица, дом 4
3850                       город Москва, 1-й переулок Тружеников, дом 18
6824                            город Москва, Дубравная улица, дом 34/29
10399                      город Москва, Новогиреевская улица, дом 31/45
11758            город Москва, Пресненская набережная, дом 4, строение 1
7872        город Москва, Сибирский проезд, дом 2, корпус 27, строение 1
3587                      город Москва, Старопименовский переулок, дом 5
12743               город Москва, улица Сущёвский Вал, дом 5, строение 5
7042                город Москва, Дмитровское шоссе, дом 108, строение 7
5404                      город Москва, проспект Мира, дом 211, корпус 2
1166                           город Москва, Тимирязевская улица, дом 19
12539        город Москва, Большая Грузинская улица

In [22]:
steet_synonyms = ['улица','ул', 'ул.', 'переулок','шоссе','проспект','площадь',
'проезд','аллея','бульвар','набережная','тупик','линия', 'просек', 'квартал', 
'микрорайон', 'территория', 'километр']

In [23]:
# test_pattern = r".*,\s*\b([^,]*?(?:{})\b[^,]*)[,$]+".format("|".join(steet_synonyms))
pattern =  r'(([а-яА-ЯёЁ0-9-й\s]+)?({})+([а-яА-ЯёЁ0-9-й\s]+)?)'.format('|'.join(steet_synonyms))

In [24]:
df["address"].str.extract(pattern, flags=re.I).head()

,0,1,2,3
0,улица Егора Абакумова,,улица,Егора Абакумова
1,улица Талалихина,,улица,Талалихина
2,Абельмановская улица,Абельмановская,улица,NaN
3,Абрамцевская улица,Абрамцевская,улица,NaN
4,Абрамцевская улица,Абрамцевская,улица,NaN


In [25]:
df["street"] = df["address"].str.extract(pattern, flags=re.I)[0]

Посмотрим, успешно ли паттерн вычленил улицы из адресов.

In [26]:
with pd.option_context('display.max_colwidth', None):
    display(df[['address', 'street']].sample(30, random_state=42))

,address,street
12535,"Вознесенский переулок, дом 7, строение 1",Вознесенский переулок
11343,"город Москва, Бартеневская улица, дом 59",Бартеневская улица
6237,"город Москва, Венёвская улица, дом 4",Венёвская улица
3850,"город Москва, 1-й переулок Тружеников, дом 18",1-й переулок Тружеников
6824,"город Москва, Дубравная улица, дом 34/29",Дубравная улица
10399,"город Москва, Новогиреевская улица, дом 31/45",Новогиреевская улица
11758,"город Москва, Пресненская набережная, дом 4, строение 1",Пресненская набережная
7872,"город Москва, Сибирский проезд, дом 2, корпус 27, строение 1",Сибирский проезд
3587,"город Москва, Старопименовский переулок, дом 5",Старопименовский переулок
12743,"город Москва, улица Сущёвский Вал, дом 5, строение 5",улица Сущёвский Вал


In [27]:
df['street'].value_counts().head(10)

 проспект Мира             196
 Профсоюзная улица         180
 Ленинградский проспект    170
 Пресненская набережная    165
 Варшавское шоссе          162
 Ленинский проспект        147
 проспект Вернадского      126
 Кутузовский проспект      114
 Каширское шоссе           110
 Кировоградская улица      107
Name: street, dtype: int64

In [28]:
df['street'].isna().sum() / df.shape[0]

0.016202331555028652

Около 1.6% наблюдений не получили улиц. Посмотрим на выборку из этих строк.

In [29]:
with pd.option_context('display.max_colwidth', None):
    display(df[['address', 'street']].query('street.isna()').sample(30, random_state=42))

,address,street
2413,"город Москва, город Зеленоград, корпус 317А, строение 1",NaN
1412,"город Москва, город Зеленоград, корпус 1449",NaN
11739,"город Москва, поселение Московский, деревня Саларьево, владение 3, строение 1",NaN
8563,"город Москва, поселение Рязановское, посёлок Знамя Октября, дом 39",NaN
10840,"город Москва, город Зеленоград, корпус 142",NaN
11736,"город Москва, поселение Московский, деревня Саларьево, владение 3, строение 1",NaN
12121,"город Москва, город Зеленоград, корпус 1805",NaN
8620,"город Москва, поселение Сосенское, деревня Сосенки, дом 126",NaN
1612,"город Москва, город Зеленоград, корпус 1805",NaN
14811,"город Москва, поселение Воскресенское, посёлок Воскресенское, дом 22А",NaN


Названия улиц не получили в основном территории за МКАДом типа Зеленограда, различных поселений и деревень, в которых не было названий улиц.

Мы не будем пытаться учесть их как улицы, так как: 
- во-первых, территории за МКАДом для робо-ресторана вряд ли будут перспективными;
- во-вторых, неправильно с точки зрения здравого смысла ставить, например, город Зеленоград в один ряд с обычными улицами. Зеленоград не улица, и в эту группу  может попасть слишком много домов;

## 4. Получение геоданных по адресу. Работа с API <a name="4"></a>

Считываем токен доступа к API. Его значение хранится в переменной окружения (в целях приватности).

In [30]:
GMAPS_TOKEN = os.environ['GMAPS_TOKEN']

In [31]:
df['address'].sample(1, random_state=42)

12535    Вознесенский переулок, дом 7, строение 1
Name: address, dtype: object

In [32]:
g_example = geocoder.google(df['address'].sample(1, random_state=42), key=GMAPS_TOKEN, language="RU")

In [33]:
g_example.json

{'accuracy': 'ROOFTOP',
 'address': 'Вознесенский пер., 7, Москва, Россия, 125009',
 'bbox': {'northeast': [55.7595139802915, 37.6050719802915],
  'southwest': [55.7568160197085, 37.6023740197085]},
 'city': 'Москва',
 'confidence': 9,
 'country': 'RU',
 'county': 'Москва',
 'housenumber': '7',
 'lat': 55.758165,
 'lng': 37.603723,
 'ok': True,
 'place': 'ChIJac8yOE9KtUYRXmjvgRXf-lA',
 'postal': '125009',
 'quality': 'street_address',
 'raw': {'address_components': [{'long_name': '7',
    'short_name': '7',
    'types': ['street_number']},
   {'long_name': 'Вознесенский переулок',
    'short_name': 'Вознесенский пер.',
    'types': ['route']},
   {'long_name': 'Центральный административный округ',
    'short_name': 'Центральный административный округ',
    'types': ['political', 'sublocality', 'sublocality_level_1']},
   {'long_name': 'Москва',
    'short_name': 'Москва',
    'types': ['locality', 'political']},
   {'long_name': 'Пресненский',
    'short_name': 'Пресненский',
    'type

In [34]:
g_example.json['raw']['address_components'][2]['short_name']

'Центральный административный округ'

In [35]:
g_example.json['raw']['address_components'][4]['short_name']

'Пресненский'

In [36]:
list(g_example.json['raw']['geometry']['location'].values())

[55.758165, 37.603723]

In [37]:
g_example.json.get('raw').get('sublocality_level_1').get('short_name')

'Центральный административный округ'

In [38]:
g_example.json.get('raw').get('administrative_area_level_3').get('short_name')

'Пресненский'

In [39]:
[g_example.json.get('lat'), g_example.json.get('lng')]

[55.758165, 37.603723]

In [40]:
g_example.latlng

In [41]:
g_example.json.get('street')

'Вознесенский пер.'

Разобрались в структуре JSON-ответа.

Теперь для отладки спарсим данные для тестовой выборки из 20 адресов (чтобы случайно не спарсить не то, что нам нужно, для всех 15 тыс. адресов).

In [42]:
address_sample = df['address'].sample(20, random_state=42)

In [43]:
address_sample.head(20)

12535                           Вознесенский переулок, дом 7, строение 1
11343                           город Москва, Бартеневская улица, дом 59
6237                                город Москва, Венёвская улица, дом 4
3850                       город Москва, 1-й переулок Тружеников, дом 18
6824                            город Москва, Дубравная улица, дом 34/29
10399                      город Москва, Новогиреевская улица, дом 31/45
11758            город Москва, Пресненская набережная, дом 4, строение 1
7872        город Москва, Сибирский проезд, дом 2, корпус 27, строение 1
3587                      город Москва, Старопименовский переулок, дом 5
12743               город Москва, улица Сущёвский Вал, дом 5, строение 5
7042                город Москва, Дмитровское шоссе, дом 108, строение 7
5404                      город Москва, проспект Мира, дом 211, корпус 2
1166                           город Москва, Тимирязевская улица, дом 19
12539        город Москва, Большая Грузинская улица

In [44]:
address_sample_df = pd.DataFrame(columns=['address', 'admin_district', 'district', 'lat_lng_geocode'])

In [45]:
for address in address_sample:
    
    g = geocoder.google(
        address,
        key=GMAPS_TOKEN, 
        language="RU"
        )

    address_sample_df.loc[len(address_sample_df.index)] = [
        address, 
        g.json['raw']['address_components'][2]['short_name'], 
        g.json['raw']['address_components'][4]['short_name'], 
        list(g.json['raw']['geometry']['location'].values())
        ]

In [46]:
address_sample_df.head(20)

,address,admin_district,district,lat_lng_geocode
0,"Вознесенский переулок, дом 7, строение 1",Центральный административный округ,Пресненский,"[55.758165, 37.603723]"
1,"город Москва, Бартеневская улица, дом 59",Юго-Западный административный округ,Южное Бутово,"[55.54617260000001, 37.51593769999999]"
2,"город Москва, Венёвская улица, дом 4",Юго-Западный административный округ,Южное Бутово,"[55.54855200000001, 37.5424999]"
3,"город Москва, 1-й переулок Тружеников, дом 18",Центральный административный округ,Хамовники,"[55.737041, 37.5715081]"
4,"город Москва, Дубравная улица, дом 34/29",Северо-Западный административный округ,Митино,"[55.846367, 37.3580737]"
5,"город Москва, Новогиреевская улица, дом 31/45",Восточный административный округ,Перово,"[55.7542051, 37.7999832]"
6,"город Москва, Пресненская набережная, дом 4, строение 1",Центральный административный округ,Пресненский,"[55.7489662, 37.5432141]"
7,"город Москва, Сибирский проезд, дом 2, корпус 27, строение 1",Центральный административный округ,Таганский,"[55.729826, 37.682961]"
8,"город Москва, Старопименовский переулок, дом 5",Центральный административный округ,Тверской,"[55.76932249999999, 37.6004401]"
9,"город Москва, улица Сущёвский Вал, дом 5, строение 5",Северо-Восточный административный округ,Марьина Роща,"[55.794884, 37.594743]"


Убедились на выборке, что и геокодер, и написанный цикл работают корректно.

Теперь можем применить аналогичный цикл к основному датафрейму. Для экономии лимита по запросам и оптимизации времени будем парсить только уникальные адреса.

In [47]:
df.shape[0]

15183

In [48]:
df['address'].unique().shape[0]

9108

In [49]:
uniq_addresses = df['address'].unique().tolist()

Ряд проверочек

In [50]:
uniq_addresses[:5]

['город Москва, улица Егора Абакумова, дом 9',
 'город Москва, улица Талалихина, дом 2/1, корпус 1',
 'город Москва, Абельмановская улица, дом 6',
 'город Москва, Абрамцевская улица, дом 1',
 'город Москва, Абрамцевская улица, дом 9, корпус 1']

In [51]:
len(uniq_addresses)

9108

In [52]:
# address_sample_df.to_csv('address_sample_df.csv', index=False)

In [53]:
GEODATA_TABLE_ID = '1rINFl9QmkSRuEKvFHxQeYUZIOd814vp5PnfItqL4FHs'
geodata_path = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(GEODATA_TABLE_ID)

try:
    geodata_r = requests.get(geodata_path)
    geodata = pd.read_csv(BytesIO(geodata_r.content))
    print('Файл прочитан из Google Sheets')
except:
    try:
        geodata = pd.read_csv('geodata.csv')
        print('Файл прочитан из CSV-файла окружения')
    except:
        geodata = pd.DataFrame(columns=['address', 'admin_district', 'district', 'lat_lng_geocode'])
        print('Сформирован пустой фрейм')

Файл прочитан из Google Sheets


In [54]:
len(geodata.index)

9094

**Код запроса к Geocoding API исполнялся около 40-50 минут. Оставляю код закоменченным. Результат парсинга сохранен в `geodata.csv` и закинут в Google Sheets, откуда теперь и подтягивается в проект (код чуть выше).**

P.S. Правильнее было бы обращаться к полученному `JSON` черещ `.get()`, а не через обычную индексацию, т.к. get понятнее и отказоустойчивее в случае, если структура JSON отличается (имеет место быть для некоторых ошибочных адресов).

In [55]:
# # счетчик для периодической перезаписи фрейма в CSV-файл,
# # чтобы не терять все данные в переменной, если ядро умрет
# counter = len(geodata.index)

# for address in uniq_addresses[len(geodata.index):]:
#     try:
#         g = geocoder.google(
#             address,
#             key=GMAPS_TOKEN, 
#             language="RU"
#             )

#         geodata.loc[len(geodata.index)] = [
#             address, 
#             g.json['raw']['address_components'][2]['short_name'], 
#             g.json['raw']['address_components'][4]['short_name'], 
#             list(g.json['raw']['geometry']['location'].values())
#             ]
#     except:
#         pass
        
    
#     # каждые 200 наполненных инфой строк фрейма перезаписываем фрейм в CSV-файл
#     counter += 1
#     if counter % 200 == 0:
#         geodata.to_csv('geodata.csv', index=False)

In [56]:
len(uniq_addresses)

9108

In [57]:
geodata.shape[0]

9094

В ходе парсинга не получили корректный JSON-ответ по 14 адресам из 9108. Не критично.

In [58]:
geodata.tail()

,address,admin_district,district,lat_lng_geocode
9089,"город Москва, Профсоюзная улица, дом 142, корпус 1, строение 1",Юго-Западный административный округ,Теплый Стан,"[55.625528, 37.509222]"
9090,"город Москва, Привольная улица, дом 11",Юго-Восточный административный округ,Выхино-жулебино,"[55.7007394, 37.8393061]"
9091,"город Москва, Салтыковская улица, дом 7Г",ул. Салтыковская,Москва,"[55.7336033, 37.8616594]"
9092,"город Москва, Осенний бульвар, дом 9",Западный административный округ,Крылатское,"[55.7582702, 37.40664750000001]"
9093,"город Москва, улица Новый Арбат, дом 13",Центральный административный округ,Арбат,"[55.752016, 37.593721]"


In [59]:
geodata['address'].duplicated().sum()

0

In [60]:
geodata['admin_district'].value_counts().head(20)

Центральный административный округ         2324
Юж. административный округ                 1043
Северо-Восточный административный округ     897
Восточный административный округ            838
Северный административный округ             821
Юго-Западный административный округ         705
Юго-Восточный административный округ        694
Западный административный округ             679
Северо-Западный административный округ      487
Новомосковский Адм. округ                   149
Зеленоград                                  108
Москва                                       65
Зеленоградский административный округ        60
Троицкий Адм. округ                          53
Ленинский р-н                                17
Подольский район                             13
Наро-Фоминский р-н                            7
Подольский р-н                                6
Троицкое                                      5
Красная Пахра                                 4
Name: admin_district, dtype: int64

Ряд адресов получил названия-синонимы административных регионов. Например, Зеленоградск и Зеленоградский АО - одно и то же. 

Не будем обрабатывать эти значения, т.к. не интересен Зеленоград. Крупнейшие АО распределены адекватно и указаны верно.

Однако посмотрим какие строки получили АО 'Москва'.

In [61]:
geodata[['admin_district', 'district']].query('admin_district == "Москва"')['district'].value_counts()

Москва            37
RU                13
Ленинский р-н      8
МО                 5
Подольский р-н     2
Name: district, dtype: int64

По ряду адресов имеем неверные значения административных районов `admin_district` и районов `district`. Вероятно, по этим адресам полученные JSON-ответы не имели корректной информации об АО и районе адреса. Возможно, в этих адресах ошибки.

Присвоим неверными значениям значение `np.NaN`.

In [62]:
wrong_districts = ['RU', 'Москва', 'МО']

In [63]:
geodata.loc[(geodata['admin_district'].isin(wrong_districts)), 'admin_district'] = np.NaN
geodata.loc[(geodata['district'].isin(wrong_districts)), 'district'] = np.NaN

In [64]:
geodata.isna().sum()

address              0
admin_district      65
district           317
lat_lng_geocode      0
dtype: int64

In [65]:
geodata['district'].value_counts().head(20)

Тверской              393
Пресненский           342
Басманный             310
Хамовники             259
Таганский             240
Замоскворечье         230
Мещанский             180
Даниловский           180
Красносельский        137
Якиманка              123
Арбат                 116
Соколиная Гора        104
Марьино               102
Южное Бутово           98
Беговой                98
Лефортово              96
Митино                 95
Хорошевский            95
Нагатино-садовники     94
Раменки                94
Name: district, dtype: int64

Убрали Москву, RU и МО из районов. Теперь районы выглядят адекватно.

Наконец, можем смерджить полученный фрейм `geodata` с основным `df` по ключу `address`.

In [66]:
df2 = df.merge(geodata, on='address', how='outer')

In [67]:
df.shape[0]

15183

In [68]:
df2.shape[0]

15183

In [69]:
df2.isna().sum()

id                   0
object_name          0
chain                0
object_type          0
address              0
number               0
street             246
admin_district     126
district           521
lat_lng_geocode     18
dtype: int64

Пропуски в улицах, АО и районе оставим. Посмотрим на пропуски в координатах `lat_lng_geocode`

In [70]:
df2.query('lat_lng_geocode.isna()')

,id,object_name,chain,object_type,address,number,street,admin_district,district,lat_lng_geocode
5943,82566,столовая квант,нет,столовая,"город Москва, город Зеленоград, проезд № 4801, дом 7, строение 1",60,проезд,NaN,NaN,NaN
6838,27515,школа 448,нет,столовая,"город Москва, посёлок Акулово, дом 43А, строение 1",156,посёлок Акулово,NaN,NaN,NaN
7245,24507,вардзия,нет,ресторан,"город Москва, шоссе Энтузиастов, домовладение 4, строение 1",250,шоссе Энтузиастов,NaN,NaN,NaN
8107,147847,новые формы,нет,ресторан,"город Москва, Кутузовский проспект, дом 12, строение 3",35,Кутузовский проспект,NaN,NaN,NaN
8108,152684,нофар,нет,ресторан,"город Москва, Кутузовский проспект, дом 12, строение 3",100,Кутузовский проспект,NaN,NaN,NaN
8367,144529,сабвей,да,кафе,"город Москва, 74-й километр Московской Кольцевой Автодороги, владение 4",8,74-й километр Московской Кольцевой Автодороги,NaN,NaN,NaN
8397,20059,якитория,да,ресторан,"город Москва, 73-й километр Московской Кольцевой Автодороги, дом 7",90,73-й километр Московской Кольцевой Автодороги,NaN,NaN,NaN
8971,20198,петрол комплекс эквипмент компани,нет,кафетерий,"город Москва, город Зеленоград, проезд № 4801, дом 3, строение 1",12,проезд,NaN,NaN,NaN
11369,158795,чайхона,нет,кафе,"город Москва, поселение ""Мосрентген"", деревня Дудкино, дом 66",42,NaN,NaN,NaN,NaN
11435,19800,набатчиков а.п.,нет,буфет,"Проектируемый проезд N 5231, дом 8, строение 3",20,Проектируемый проезд,NaN,NaN,NaN


Ряд адресов типа 'проектируемый проезд' не гуглится. Их, вероятно, по крайней мере на данный момент не существует. 

Все эти адреса не получили корректной информации ни по координатам, ни по району, ни по АО. Удалим эти строки.

In [71]:
df2 = df2.query('lat_lng_geocode.notna()')

In [72]:
df2.isna().sum()

id                   0
object_name          0
chain                0
object_type          0
address              0
number               0
street             240
admin_district     108
district           503
lat_lng_geocode      0
dtype: int64

In [73]:
df2.sample(10, random_state=42)

,id,object_name,chain,object_type,address,number,street,admin_district,district,lat_lng_geocode
3691,156171,донерок,нет,кафе,"город Москва, Кронштадтский бульвар, дом 3, строение 3",0,Кронштадтский бульвар,Северный административный округ,Головинский,"[55.840864, 37.484727]"
12287,170678,донер №1,нет,предприятие быстрого обслуживания,"город Москва, Ботаническая улица, дом 41, корпус 7",0,Ботаническая улица,Северо-Восточный административный округ,Марфино,"[55.846276, 37.583892]"
12019,163950,столовая,нет,столовая,"город Москва, Алтуфьевское шоссе, дом 31, строение 1",80,Алтуфьевское шоссе,Северо-Восточный административный округ,Отрадное,"[55.861476, 37.580076]"
4629,169706,тажинерия,нет,кафе,"город Москва, Усачёва улица, дом 26",12,Усачёва улица,Центральный административный округ,Хамовники,"[55.7272093, 37.5690324]"
13953,190895,магбургер,да,ресторан,"город Москва, поселение Щаповское, деревня Троицкое, дом 3Б/Н",24,NaN,Троицкое,NaN,"[55.3881616, 37.4195508]"
3248,24857,бухарест,нет,кафе,"город Москва, улица Каховка, дом 27",70,улица Каховка,Юго-Западный административный округ,Зюзино,"[55.655364, 37.5720159]"
2861,167297,ресторан «cut»,нет,ресторан,"город Москва, Большая Никитская улица, дом 14/2, строение 7",40,Большая Никитская улица,Центральный административный округ,Пресненский,"[55.7567331, 37.605561]"
6631,150878,вгик,нет,столовая,"город Москва, улица Вильгельма Пика, дом 3, строение 1",200,улица Вильгельма Пика,Северо-Восточный административный округ,Ростокино,"[55.834543, 37.638318]"
14889,206181,дом татарской кухни,нет,кафе,"город Москва, улица Касаткина, дом 13, строение 9",20,улица Касаткина,Северо-Восточный административный округ,Алексеевский,"[55.826119, 37.665587]"
10431,24924,кружка,да,кафе,"город Москва, Ладожская улица, дом 5",95,Ладожская улица,Центральный административный округ,Басманный,"[55.7716129, 37.6821536]"


## 5. Обогащение данных информацией о населении районов <a name="5"></a>

В идеале для целей исследования было бы хорошо подтянуть информацию о потоке людей по адресу, однако такие данные:
- либо стоят денег;
- либо вовсе отсутствуют и требуют сбора (например, гео-аналитические услуги на аутсорсе, что может быть очень дорого).

В качестве достаточно грубой аппроксимации потока людей подгрузим данные о населении районов и смерджим их с нашими. Данные о населении районов взяты с сайта ФСГС по г. Москве и МО, отредактированы под нужный вид и залиты на Google Sheets, откуда мы и будем их читать.

Ссылка на источник:
https://mosstat.gks.ru/folder/64634. Файл: *Оценка численности постоянного населения г.Москвы на 1 января 2020 года*.

In [74]:
DISTRICTS_POP_TABLE_ID = '1S0KKKtgHCV7A8e1pKyReD25NmkTCL_pUWkZG3QeOQls'
pop_path = 'https://docs.google.com/spreadsheets/d/{}/export?format=csv'.format(DISTRICTS_POP_TABLE_ID)
pop_r = requests.get(pop_path)
pop_df = pd.read_csv(BytesIO(pop_r.content))

In [75]:
print(pop_df.shape)
pop_df.head()

(173, 2)


,district,pop
0,Восточный округ,1527316.0
1,в том числе муниципальные образования Восточного округа:,NaN
2,Богородское,110137.0
3,Вешняки,122419.0
4,Восточное Измайлово,78206.0


In [76]:
pop_df = pop_df.query('district.str.contains("округ", case=False)==False').sort_values(by='district')

In [77]:
pop_df.head()

Проверим, одинаково ли записаны названия районов в подгруженной информации о популяции и наших данных.

In [78]:
uniq_districts = df2['district'].dropna().unique()

In [79]:
uniq_districts[:5]

array(['Ярославский', 'Таганский', 'Лианозово', 'Арбат', 'Лефортово'],
      dtype=object)

In [80]:
district_names_diff = sorted(list(set(pop_df['district']).symmetric_difference(uniq_districts)))

In [81]:
district_names_diff[:10]

['Академический',
 'Академическое',
 'Алексеевский',
 'Алексеевское',
 'Алтуфьевский',
 'Алтуфьевское',
 'Бабушкинский',
 'Бабушкинское',
 'Басманное',
 'Басманный']

Имеем разные окончания для названий районов в `pop_df` и `df2`. Это не позволит нам смерджить эти таблички по району корректно.

Приведем все названия районов к их нормальной форме.

In [82]:
morph = pymorphy2.MorphAnalyzer()

for word in district_names_diff[:10]:
    print(morph.parse(word)[0].normal_form)

академический
академический
алексеевский
алексеевский
алтуфьевский
алтуфьевский
бабушкинский
бабушкинский
басманный
басманный


In [83]:
pop_df['morph_district'] = pop_df['district'].map(lambda x: morph.parse(x)[0].normal_form)

In [84]:
pop_df.head()

,district,pop,morph_district
115,Академическое,110038.0,академический
60,Алексеевское,80634.0,алексеевский
61,Алтуфьевское,57697.0,алтуфьевский
89,Арбат,36308.0,арбат
42,Аэропорт,79541.0,аэропорт


In [85]:
df3 = df2.copy()

In [86]:
df3['morph_district'] = (
    df3['district']
    .replace(np.nan, 'nan')
    .map(lambda x: morph.parse(x)[0].normal_form)
    .replace('nan', np.nan)
    )

In [87]:
df3[['district','morph_district']].sample(20, random_state=42)

,district,morph_district
3691,Головинский,головинский
12287,Марфино,марфино
12019,Отрадное,отрадный
4629,Хамовники,хамовник
13953,NaN,NaN
3248,Зюзино,зюзино
2861,Пресненский,пресненский
6631,Ростокино,ростокино
14889,Алексеевский,алексеевский
10431,Басманный,басманный


In [88]:
df3.merge(pop_df, on='morph_district', how='inner').shape[0] / df3.shape[0]

0.9186943620178042

После обработки, около 92% записей в `df3` имеют названия районов, соответствующие названиям из `pop_df`.

Будем использовать **LEFT JOIN** и, соответственно, получим около 8% записей с пропусками в столбце населения района `pop`. Приемлемо.

In [89]:
df3 = df3.merge(pop_df[['morph_district', 'pop']], on='morph_district', how='left')

In [90]:
df3[['morph_district', 'district', 'pop']].sample(20, random_state=42)

,morph_district,district,pop
3691,головинский,Головинский,103573.0
12276,марфино,Марфино,35579.0
12008,отрадный,Отрадное,185745.0
4629,хамовник,Хамовники,109451.0
13938,NaN,NaN,NaN
3248,зюзино,Зюзино,127301.0
2861,пресненский,Пресненский,128314.0
6630,ростокино,Ростокино,40196.0
14873,алексеевский,Алексеевский,80634.0
10423,басманный,Басманный,110897.0


In [91]:
df3 = df3.drop(columns='morph_district')

In [92]:
'morph_district' in df3.columns

False

## 6. EDA <a name="6"></a>

Согласно поставленным задачам, нужно исследовать:

- соотношение видов объектов общественного питания по количеству;
- соотношение сетевых и несетевых заведений по количеству;
- для какого вида объекта общественного питания характерно сетевое распространение;
- что характерно для сетевых заведений: много заведений с небольшим числом посадочных мест в каждом или мало заведений с большим количеством посадочных мест;
- среднее количество посадочных мест для каждого вида объекта общественного питания. Какой вид предоставляет в среднем самое большое количество посадочных мест;
- топ-10 улиц по количеству объектов общественного питания, а также в каких районах Москвы находятся эти улицы;
- число улиц с одним объектом общественного питания. В каких районах Москвы находятся эти улицы;
- распределение количества посадочных мест для улиц с большим количеством объектов общественного питания, описать закономерности.

Идем по порядку. В конце раздела попытаемся дать окончательный результат анализа с использованием информации о населении районов.

In [93]:
object_types_counts = df3['object_type'].value_counts()

In [94]:
fig = px.bar(
    object_types_counts,
    text=object_types_counts,
    color=object_types_counts.index,
    labels={'index':'object_type', 'value':'count'},
    title='Наиболее популярные категории объектов общественного питания'
    )

fig.update_layout(showlegend=False)

- категория кафе лидирует с отрывом (~40% всех объектов общественного питания - кафе);
- на четверку категорий-лидеров приходится ~80% всех объектов общественного питания;

Предполагается, что наш робо-ресторан как раз будет кафе — самым популярным форматом заведений.

In [95]:
chain_object_type_report = (
    df3
    .groupby(['object_type', 'chain'])
    .agg({'id':'count'})
    .reset_index()
    .rename(columns={'id':'is_chain_count'})
    )

In [96]:
chain_object_type_report.head()

,object_type,chain,is_chain_count
0,бар,да,37
1,бар,нет,815
2,буфет,да,11
3,буфет,нет,557
4,закусочная,да,56


In [97]:
object_type_g = (
    df3
    .groupby(['object_type'])
    .agg({'id':'count'})
    .reset_index()
    .rename(columns={'id':'count'})
    )

In [98]:
object_type_g.head()

,object_type,count
0,бар,852
1,буфет,568
2,закусочная,347
3,кафе,6004
4,кафетерий,392


In [99]:
chain_object_type_report = chain_object_type_report.merge(object_type_g, on='object_type')

In [100]:
chain_object_type_report['is_chain_share'] = (chain_object_type_report['is_chain_count'] / chain_object_type_report['count']).round(3)

In [101]:
chain_object_type_report.head()

,object_type,chain,is_chain_count,count,is_chain_share
0,бар,да,37,852,0.043
1,бар,нет,815,852,0.957
2,буфет,да,11,568,0.019
3,буфет,нет,557,568,0.981
4,закусочная,да,56,347,0.161


In [102]:
fig = px.bar(
    (
        chain_object_type_report
        .query('chain == "да"')
        .sort_values(by='is_chain_share', ascending=False)
    ), 
    x='object_type', 
    y='is_chain_share', 
    color='object_type',
    text='is_chain_share',
    labels={
        'is_chain_share':'Доля сетевых объектов',
        'object_type':'Категория объекта'
        },
    title='Наибольшая доля сетевых объектов общественного питания по категориям'

    )

fig.update_traces(texttemplate='%{text:.1%}')
fig.layout.yaxis.tickformat = ',.0%'
fig.update_layout(showlegend=False)

- вполне интуитивно, что предприятия быстрого обслуживания имеют наивысшую долю сетевых заведений (можно было ожидать долю еще выше);
- интересно, что для баров крайне нехарактерно сетевое распространение;

Так или иначе, вряд ли для открытия робо-ресторана получится опираться на уже готовую франшизу.

In [103]:
px.box(
    df3, 
    x='number', 
    color='chain',
    labels={'number':'Количество посадочных мест', 'chain':'Сеть'},
    title='Характерное количество посадочных мест (сетевые заведения против несетевых)'
    )

- медианное значение для обеих групп - 40 мест;
- для сетевых заведений закономерно видим меньший разброс количества посадочных мест — сетевые франшизы более уницифированы по формату, включая, вероятно, и количество посадочных мест;
- несетевые заведения более разнообразны по этому показателю, а также выделяются выбросами выше 600 посадочных мест - бары в клубах, крупные рестораны и прочее;

In [104]:
top10_streets = (
    df3
    .groupby('street')
    .agg({'object_name':'count', 'number':'median'})
    .sort_values(by='object_name', ascending=False).head(10)
    .rename(columns={'object_name':'n_objects', 'number':'n_seats'})
    .reset_index()
    )

In [105]:
top10_streets

,street,n_objects,n_seats
0,проспект Мира,196,45.0
1,Профсоюзная улица,180,24.5
2,Ленинградский проспект,170,40.0
3,Пресненская набережная,165,30.0
4,Варшавское шоссе,162,30.0
5,Ленинский проспект,147,45.0
6,проспект Вернадского,126,40.0
7,Кутузовский проспект,112,40.0
8,Каширское шоссе,110,25.0
9,Кировоградская улица,107,30.0


In [106]:
px.bar(
    top10_streets, 
    x='street', 
    y=['n_objects', 'n_seats'], 
    barmode='group', 
    title='Топ-10 улиц по количеству объектов и характерное для них количество посадочных мест',
    labels={'n_objects':'Количество объектов', 'n_seats':'Количество посадочных мест'})

Не видно закономерности между количеством объектов на улице и медианным количеством посадочных мест.

Можно было бы ожидать, что чем популярнее улица, тем более низкое медианное количество посадочных мест можно наблюдать для такой улицы (в первую очередь из-за дорогой аренды), однако для более обоснованной проверки такой гипотезы нужно как минимум нормировать количество объектов на улице на протяженность этих улиц. Для этого вновь пришлось бы обогощать данные, однако такая информация не несет большого значения для целей исследования, поэтому мы этого делать не будем.

Проверим согласно заданию, в каких районах Москвы находятся эти улицы.

In [107]:
df3.query('street in @top10_streets.street')['district'].unique()

array(['Донской', 'Нагорный', 'Чертаново Южное', 'Гагаринский',
       'Ломоносовский', 'проспект Вернадского', 'Тропарево-никулино',
       'Фили-давыдково', 'Нагатино-садовники', 'Москворечье-сабурово',
       'Чертаново Центральное', 'Дорогомилово', 'Беговой', 'Аэропорт',
       'Сокол', 'Якиманка', 'Обручевский', 'Мещанский', 'Останкинский',
       'Алексеевский', 'Академический', 'Черемушки', 'Коньково',
       'Теплый Стан', 'Раменки', 'Орехово-борисово Южное', nan,
       'Чертаново Северное', 'Хорошевский', 'Орехово-борисово Северное',
       'Ростокино', 'Свиблово', 'Ясенево', 'Южное Бутово', 'Пресненский',
       'Ярославский'], dtype=object)

Не очень информативно, т.к. одна улица может входить в большое количество районов. 

Посмотрим на самые популярные улицы (а точнее на объекты на этих улицах) на карте для большей наглядности.

In [108]:
df3['lat_lng_geocode'][0]

'[55.8792377, 37.7141957]'

При парсинге координаты записались как строки `str` вместо листов `list`. Исправим это и перезапишем широту и долготу как отдельные `float` столбцы, т.к. с серией листов не удобно работать.

P.S. My bad. Мог изначально так сделать при получении ответов от API по адресам.

In [109]:
# это превратит значения столбца из str в list
df3['lat_lng_geocode'] = df3['lat_lng_geocode'].map(eval)

In [110]:
df3['lat_lng_geocode'][0]

[55.8792377, 37.7141957]

In [111]:
df3['lat'] = df3['lat_lng_geocode'].map(lambda x: x[0])
df3['lng'] = df3['lat_lng_geocode'].map(lambda x: x[1])

In [112]:
df3[['lat_lng_geocode', 'lat', 'lng']].head()

,lat_lng_geocode,lat,lng
0,"[55.8792377, 37.7141957]",55.879238,37.714196
1,"[55.738252, 37.6733781]",55.738252,37.673378
2,"[55.7355, 37.669608]",55.735500,37.669608
3,"[55.8917906, 37.5734207]",55.891791,37.573421
4,"[55.9049383, 37.572052]",55.904938,37.572052


In [113]:
df3 = df3.drop(columns='lat_lng_geocode')

Гуглим координаты центра Москвы

In [114]:
msk_center = {'lat': 55.751244, 'lng': 37.618423}

In [115]:
px.scatter_mapbox(
    df3.query('street in @top10_streets.street').dropna().sort_values(by='street', ascending=False), 
    lat='lat',
    lon='lng',
    center=dict(lat=msk_center['lat']-0.07, lon=msk_center['lng']), 
    zoom=9.5,
    height=800,
    color='street',
    mapbox_style='carto-positron',
    title='Топ-10 улиц Москвы по количеству объектов общественного питания'
    )

Видим, что в топ-10 улиц, действительно, попали по большей части попросту самые длинные улицы. Как мы уже отметили, для определенния популярных улиц нужно количество заведений на улице нормировать на протяженность улицы, чего мы делать не будем ввиду невысокой пользы такой информации для целей исследования.

Согласно заданию посмотрим на улицы с одним объектом общественного питания.

In [116]:
streets_w_one_object = df3.groupby('street').agg({'object_name':'count'})

In [117]:
streets_w_one_object = streets_w_one_object[streets_w_one_object <= 1].dropna().index.tolist()

In [118]:
len(streets_w_one_object)

606

В Москве целых 606 улиц с 1 объектом общественного питания.

Тем не менее, эта информация ничего нам не дает.

In [119]:
px.scatter_mapbox(
    df3.query('street in @streets_w_one_object').dropna(), 
    lat='lat',
    lon='lng',
    center=dict(lat=msk_center['lat']-0.05, lon=msk_center['lng']), 
    zoom=9,
    height=800,
    mapbox_style='carto-positron',
    title='Топ-10 улиц Москвы по количеству объектов общественного питания'
    )

Очевидно, что все эти 606 улиц с одним объектом общественного питания распределены по всей Москве (вплоть до Зеленограда), а не в каких-то особенных районах.

С заданиями разобрались. Теперь небольшое доп. исследование. Данных о потоке людей, как мы уже сказали, у нас нет, поэтому будем опираться на население района. 

Исследуем, какая доля населения района обеспечена посадочными местами.

In [120]:
final_report = (
    df3
    .query('object_type == "кафе"')
    .groupby('district')
    .agg({
        'lat':'mean', 
        'lng':'mean', 
        'object_name':'count', 
        'number':'sum', 
        'pop':'mean'
        })
    .reset_index()
    .sort_values(by='pop', ascending=False)
    .dropna()
    .rename(columns={
        'lat':'centroid_lat',
        'lng':'centroid_lng',
        'object_name':'n_objects',
        'number':'n_seats'
        })
    )

In [121]:
final_report.head()

,district,centroid_lat,centroid_lng,n_objects,n_seats,pop
62,Марьино,55.654217,37.746129,64,1647,254142.0
22,Выхино-жулебино,55.698930,37.833422,40,1565,225493.0
136,Южное Бутово,55.542004,37.536439,28,1585,210783.0
66,Митино,55.845902,37.362984,43,1664,195476.0
85,Отрадное,55.861476,37.596333,55,3464,185745.0


In [122]:
final_report['seats_per_pop'] = (final_report['n_seats'] / final_report['pop']).round(3)

In [123]:
final_report.head()

,district,centroid_lat,centroid_lng,n_objects,n_seats,pop,seats_per_pop
62,Марьино,55.654217,37.746129,64,1647,254142.0,0.006
22,Выхино-жулебино,55.698930,37.833422,40,1565,225493.0,0.007
136,Южное Бутово,55.542004,37.536439,28,1585,210783.0,0.008
66,Митино,55.845902,37.362984,43,1664,195476.0,0.009
85,Отрадное,55.861476,37.596333,55,3464,185745.0,0.019


In [124]:
final_report['seats_per_pop'].describe()

count    121.000000
mean       0.022207
std        0.036716
min        0.001000
25%        0.006000
50%        0.010000
75%        0.020000
max        0.268000
Name: seats_per_pop, dtype: float64

In [125]:
px.scatter_mapbox(
    final_report,
    lat='centroid_lat',
    lon='centroid_lng',
    center=dict(lat=msk_center['lat'], lon=msk_center['lng']), 
    zoom=9,
    color='seats_per_pop',
    range_color=[0, 0.075],
    text='district',
    size='pop',
    size_max=30,
    color_continuous_scale=px.colors.diverging.RdYlGn,
    height=800,
    mapbox_style='carto-positron',
    title='Обеспеченность населения района посадочными местами в кафе'
    )

**Зеленые центральные районы имеют наибольший поток людей. Красные — наименьший. Открывать робо-ресторан лучше в зеленых районах с большим потоком. Естественной внешней границей для открытия может выступить Садовое кольцо. В него входят районы: Мещанскй, Тверской, Арбат, Якиманка, Замоскворечье, Басманный. **

- на центральные районы приходится избыточное количество посадочных мест на население района из-за того, что в центре имеем большой туристический (и не только) поток людей;
- из-за того, что робо-кафе —  весьма специфичный формат, он будет являться аналогом достопримечательности, которой требуется постоянный поток новых людей. Именно поэтому некоторые спальные районы с крупным населением и небольшим количеством кафе **не подойдут** —  нас интересует максимальный охват новых посетителей, а не попытка в удержание покупателей;

In [126]:
good_districts = ['Мещанский', 'Тверской', 'Арбат', 'Якиманка', 'Замоскворечье','Басманный']

In [127]:
px.scatter_mapbox(
    final_report.query('district in @good_districts'),
    lat='centroid_lat',
    lon='centroid_lng',
    center=dict(lat=msk_center['lat']-0.005, lon=msk_center['lng']), 
    zoom=12,
    color='district',
    range_color=[0, 0.075],
    size='pop',
    size_max=30,
    height=800,
    mapbox_style='carto-positron',
    title='Перспективные районы для открытия робо-кафе'
    )

In [128]:
px.box(
    df3.query('district in @good_districts'), 
    x='number',
    labels={'number':'Количество посадочных мест'},
    title='Характерное количество посадочных мест внутри Садового кольца'
    )

Для выбранных нами районов адекватным количеством посадочных мест выглядит значение между медианой и верхним квартилем (от 45 до 80 мест). Мы не рассматриваем нижний квартиль из-за того, т.к. для дорогого робо-кафе в меньшей степени подходит формат бюджетной миниатюрной кафешки.

## 7. Результаты исследования <a name="7"></a>

- наиболее распространенный формат объектов общественного питания — кафе. Нашему робо-заведению следует рассмотреть именно этот формат в первую очередь;
- оптимальное количество посадочных мест от 45 до 80;
- оптимальные районы — районы внутри Садового кольца с большим туристическим потоком людей: Мещанский, Тверской, Арбат, Якиманка, Замоскворечье и Басманный.

Презентация для инвесторов: https://drive.google.com/file/d/1fy85g-DAT6_jlOGLcQzLXicaNeM_QFLo/view?usp=sharing